In [ ]:
import sys
import os
# go to upper diretory
sys.path.append(os.path.abspath('./../../../'))
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
import librosa
from sklearn.ensemble import RandomForestClassifier as RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, KFold
from Audio_Sentiment_Analysis.utils.Configuration import Configuration
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanAbsoluteError, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

AUDIO_DIR = f"{os.path.abspath('./../../../')}/IEMOCAP_Dataset"
EXTRACTED_FEATURES_FILE = f"{os.path.abspath('./../../../')}/Audio_Sentiment_Analysis/iemocap/data/extracted_features_iemocap.csv"
CONFIG_FILE = f"{os.path.abspath('./../../../')}/Audio_Sentiment_Analysis/iemocap/config.json"
RAW_AUDIO_FILES = f"{os.path.abspath('./../../../')}/Audio_Sentiment_Analysis/iemocap/data/raw_audio_files.csv"

config = Configuration.load_json(CONFIG_FILE)
# !pip install autokeras
# !pip install --upgrade scipy==1.7.0
# !pip install auto-sklearn
import autokeras as ak
# from autosklearn.classification import AutoSklearnClassifier
# from autosklearn.regression import AutoSklearnRegressor
plt.rcParams['figure.dpi'] = 300

## Read the extracted features from the CSV

In [ ]:
df = pd.read_csv(EXTRACTED_FEATURES_FILE)
print(f"Number of Audio Files: {df.shape[0]}")
df = df.sort_values(['Emotion_Id', 'Gender'], ascending = (True, True))
df = df.set_index('File')
df

## Data used in SOA models

In [ ]:
df = df[df['Emotion'].isin({'angry', 'neutral', 'sad', 'happy', 'excited'})]
df.loc[df['Emotion'] == 'excited', 'Emotion'] = 'happy'
df.loc[df['Emotion_Id'] == 5, 'Emotion_Id'] = 1
data = df.iloc[:,8:]
regression_labels = df.iloc[:,5:8].values
df.groupby(['Emotion', 'Emotion_Id']).agg({'Emotion': ['count']})

# Regression Problem (Valence, Activation, Dominance)

## Models

### Simple Random Forests

In [ ]:
pred_values = cross_val_predict(RandomForestRegressor(random_state=1, max_features=None),
    data.values, regression_labels, cv=5, verbose=1, n_jobs=8)
print('MAE: %.3f' % metrics.mean_absolute_error(regression_labels, pred_values))
print('MSE: %.3f' % metrics.mean_squared_error(regression_labels, pred_values))
print('R2: %.3f' % metrics.r2_score(regression_labels, pred_values))
print('EV: %.3f' % metrics.explained_variance_score(regression_labels, pred_values))

### AutoSKlearn

In [ ]:
model = AutoSklearnRegressor(
    max_models_on_disc=None,
    resampling_strategy='cv',
    resampling_strategy_arguments={'cv': {'folds': 5}},
    seed=1,
    n_jobs=-1
)
model.fit(data.values, regression_labels)

In [ ]:
print(model.sprint_statistics())

In [ ]:
model.show_models()

In [ ]:
model.leaderboard()

#### Testing the Highest Ranked Model from the Auto Keras

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
pred_values = cross_val_predict(ExtraTreesRegressor(max_features=0.6058051610484844, min_samples_leaf=9,
    min_samples_split=10, n_estimators=512, n_jobs=8, random_state=1, warm_start=True),
    data.values, regression_labels, cv=5, verbose=1, n_jobs=8)
print('MAE: %.3f' % metrics.mean_absolute_error(regression_labels, pred_values))
print('MSE: %.3f' % metrics.mean_squared_error(regression_labels, pred_values))
print('R2: %.3f' % metrics.r2_score(regression_labels, pred_values))
print('EV: %.3f' % metrics.explained_variance_score(regression_labels, pred_values))

### Convolution Neural Networks

In [ ]:
def build_model_0(input_shape=(24, 1), loss=MeanSquaredError(), optimizer=Adam(learning_rate=1e-5, epsilon=1e-6)):
    if loss.__class__ == SparseCategoricalCrossentropy().__class__:
        n_labels = 4
        metrics = ['accuracy']
        activation = 'softmax'
    else:
        n_labels = 3
        metrics = ['mae']
        activation = 'relu'

    model = Sequential()
    model.add(Conv1D(256, (5), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv1D(128, (5)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(n_labels, strides=1))

    model.add(Conv1D(128, (5)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(n_labels))

    model.add(Activation(activation))

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model
build_model_0(input_shape=(data.iloc[0].shape[0],1)).summary(show_trainable=True)

In [ ]:
def regression_cross_validation(callbacks, no_epochs, batch_size, loss, optimizer, verbosity, num_folds):
  print('------------------------------------------------------------------------')
  kfold = KFold(n_splits=num_folds, shuffle=True, random_state=1)
  mae_per_fold = []
  loss_per_fold = []
  fold_no = 1
  
  X = df.iloc[:,8:]
  y = df.iloc[:,5:8]

  for train, test in kfold.split(X, y):
    model = build_model_0(input_shape=(data.shape[1], 1), loss=loss, optimizer=optimizer)
    
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

    print(f'Training for fold {fold_no} ...')

    history = model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=no_epochs,
                verbose=verbosity,
                callbacks=callbacks,
                workers=4)

    scores = model.evaluate(X_test, y_test, verbose=verbosity)

    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    mae_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(mae_per_fold)):
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - MAE: {mae_per_fold[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> MAE: {np.mean(mae_per_fold)} (+- {np.std(mae_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  print('------------------------------------------------------------------------')

In [ ]:
callback = EarlyStopping(monitor='loss', patience=10)
no_epochs = 500 # try 300 or 700
batch_size = 32 # try 16
learning_rate=1e-5 # try 1e-5
loss = MeanAbsoluteError()
optimizer = Adam(learning_rate=learning_rate)

verbosity = 0
num_folds = 5

regression_cross_validation(callback, no_epochs, batch_size, loss, optimizer, verbosity, num_folds)

### AutoKeras

In [ ]:
def categorical_cross_validation2(no_epochs, batch_size, loss, optimizer, verbosity, num_folds):
  print('------------------------------------------------------------------------')
  kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)
  acc_per_fold = []
  loss_per_fold = []
  fold_no = 1
  
  X = df.iloc[:,8:]
  y = df.iloc[:,4:5]

  for train, test in kfold.split(X, y):
    model = ak.StructuredDataClassifier(overwrite=True, max_trials=100, seed=1, directory=f"best_regression_keras_model_fold_{fold_no}")
    
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

    print(f'Training for fold {fold_no} ...')

    history = model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=no_epochs,
                verbose=verbosity,
                callbacks=[EarlyStopping()],
                workers=8)
  
    print(model.export_model().summary())

    scores = model.evaluate(X_test, y_test, verbose=verbosity)

    print(f'Score for fold {fold_no}: Loss of {scores[0]}; Accuracy of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  print('------------------------------------------------------------------------')

# classifier that tries 100 different keras classifier models
no_epochs = 500 # try 300 or 700
batch_size = 16 # try 16
learning_rate=1e-5 # try 1e-4
loss = SparseCategoricalCrossentropy()
optimizer = Adam(learning_rate=learning_rate)

verbosity = 1
num_folds = 5

categorical_cross_validation2(no_epochs, batch_size, loss, optimizer, verbosity, num_folds)

#### Testing with Best Model

In [ ]:
def categorical_cross_validation4(model, verbosity, num_folds):
  print('------------------------------------------------------------------------')
  kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)
  acc_per_fold = []
  loss_per_fold = []
  fold_no = 1

  X = df.iloc[:,8:]
  y = df.iloc[:,4:5]

  for train, test in kfold.split(X, y):    
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

    y_test = y_test.astype('str')
    y_test = OneHotEncoder().fit_transform(y_test).toarray()
    print(y_test)

    scores = model.evaluate(X_test, y_test, verbose=verbosity)

    print(f'Score for fold {fold_no}: Loss of {scores[0]}; Accuracy of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
      print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  print('------------------------------------------------------------------------')

## Mel-Spectogram as the Input Feature